# Extract the grid boxes (individual images) from the larger grid image used for handwriting data generation

## Directory in which the images/data would be extracted to

In [33]:
dir_name = 'mnist_plus_png/9'

## Extract the images from the grid and perform processing

In [34]:
from PIL import Image
import cv2
import numpy as np
import image_slicer
# import torchvision, torch

filename_ = dir_name+'/9_grid_225'
filename = filename_+'.png'
img = Image.open(filename, mode='r')



image_slicer.slice(filename, 225)

for i in range(1,16,1):
    for j in range(1,16,1):
        tilename = filename_+'_'+str(i).zfill(2)+'_'+str(j).zfill(2)+'.png'
#         print(tilename)
        tileImg = Image.open(tilename, mode='r')
        w, h = tileImg.size
        tileImg = tileImg.crop((2, 2, w-2, h-2))
        tileImg.save(tilename)
        
        ###### Centering begin
        # Load image as grayscale and obtain bounding box coordinates
        image = cv2.imread(tilename, 0)

        height, width = image.shape
        x,y,w,h = cv2.boundingRect(image)

        # Create new blank image and shift ROI to new coordinates
        ROI = image[y:y+h, x:x+w]
        mask = np.zeros([ROI.shape[0]+10,ROI.shape[1]+10])
        width, height = mask.shape
        #     print(ROI.shape)
        #     print(mask.shape)
        x = width//2 - ROI.shape[0]//2 
        y = height//2 - ROI.shape[1]//2 
        #     print(x,y)
        mask[y:y+h, x:x+w] = ROI

        output_image = Image.fromarray(mask)
        compressed_output_image = output_image.resize((22,22))
        
        image_array = np.array(compressed_output_image.getdata()).reshape(22,22)
#         print(image_array.shape)
        image_array = np.pad(image_array, (3,3),'constant', constant_values=(0,0))
        padded_output_image = Image.fromarray(image_array)
        padded_output_image = padded_output_image.convert("L")
        padded_output_image.save(tilename)
#         convert_tensor = torchvision.transforms.ToTensor() # Normalization
#         tensor_image = convert_tensor(compressed_output_image) 
#         tensor_image = torch.nn.functional.pad(tensor_image, (3,3,3,3), "constant", 0) # Padding
#         # Standardization should be done after padding i guess
#         convert_tensor = torchvision.transforms.Normalize((0.1307), (0.3081)) # Standardization (mean and std from MNIST)
#         tensor_image = convert_tensor(tensor_image)
#         # plt.imshow(tensor_image.detach().cpu().numpy().reshape(28,28), cmap='viridis')

#         processed_image = Image.fromarray(tensor_image.detach().cpu().numpy().reshape(28,28))
#         processed_image = processed_image.convert("L")
#         processed_image.save(tilename)
        